In [1]:
using Pkg;Pkg.activate(".");Pkg.instantiate()
using Distributions, Zygote, PDMats
include("utils.jl")

 Activating environment at `~/biaslab/repos/kalman_filters/Project.toml`


posterior (generic function with 1 method)

## Making sense of Expected Free Energy
This notebook is a demo of EFE in a Kalman filter-like model. Also currently a showcase of bad writing, placeholders and tests of how to write a proper demo

Starting point is the original 2015 EFE paper. In it Friston defines EFE as
\begin{align}
EFE = \iint \widetilde{q}(o,s|\pi) \log \frac{q(s|\pi)}{p(o,s|\pi)} \mathrm{d}o \mathrm{d}s
\end{align}
However taking a closer look at the implementation, we notice a few details. The model in question is the, by now, canonical T-maze experiment. It is defined on a discrete state spaces in terms of 5 matrices, 4 of which are relevant for this demo
\begin{align}
p(o_t|s_t) = A\\
p(s_t | s_{t-1}, u_t) = B(u_t)\\
p(o_t|m) = C\\
p(s_0 | m) = D
\end{align}
Here $A$ encodes the likelihood models, $B(u_t)$ a state transition that depends on action $u_t$ and $D$ a belief over the initial state. All of these can be recovered from the canonical EFE derivation in Appendix A. However the matrix $C$ does not feature in the derivation in the appendix, so it deserves a special mention. It denotes a prior over future outcomes in terms of their __utility__.\
Quote "The priors over future outcomes specify their utility $C(o_t|m) = p(o_t|m) = ln C_t$". This matrix specifies a cost associated with each state and is the driving term behind goal directed behaviour. If we incorporate $p(o|m)$ into the derivation of EFE we end up with an additional term. This term is not unknown in AI literature and has for instance been seen in \cite{thijs} as a "goal prior". We write
\begin{align}
EFE &= \iint \widetilde{q}(o,s|\pi) \log \frac{q(s|\pi)}{p(o,s|\pi)p(o|m)} \mathrm{d}o \mathrm{d}s\\
&\iint \widetilde{q}(o,s|\pi) \log \frac{q(s|\pi)}{\bar{p}(o,s|\pi)} \mathrm{d}o \mathrm{d}s\\
&\bar{p}(o,s | \pi) = \frac{1}{Z}p(o,s |\pi)p(o|m)
\end{align}
\
Where in the last line we utilize the same definition as \cite{generalised FE 2019}. Decomposing this EFE we find
\
\begin{align}
EFE &= \iint \widetilde{q}(o,s|\pi) \log \frac{q(s|\pi)}{\bar{p}(o,s|\pi)} \mathrm{d}o \mathrm{d}s\\
&=\iint \widetilde{q}(o,s|\pi) \log \frac{q(s|\pi)}{p(o,s|\pi)} - \log p(o|m)\mathrm{d}o \mathrm{d}s\\
&=\iint \widetilde{q}(o,s|\pi) \log \frac{q(s|\pi)}{p(o,s|\pi)} \mathrm{d}o \mathrm{d}s - \iint \widetilde{q}(o,s|\pi) \log p(o|m)\mathrm{d}o \mathrm{d}s\\
&= H[\widetilde{q}(o,s|\pi)||p(o,s|\pi)] -H[q(s | \pi)] - \mathbf{E}_{\widetilde{q}(o,s|\pi)} [C_t]
\end{align}
\
where we recognise that EFE - with all terms used in the experiments included - decompose into an entropy term, a crossentropy and an expected utility or reward term relying on an arbitrary cost function. This cost is usually interpreted as a preference over future outcomes or a goal prior.\
The next step is to examine the defition of $\widetilde{q}(o,s|\pi)$. This is given in Appendix A of \cite{efe paper} as
\begin{align}
\widetilde{q}(o,s|\pi) = p(o|s)q(s|\pi)
\end{align}
However upon examination of the simulations this turns out to denote that the same matrix $A$ is used as the likelihood for both generative and recognition densities. We are therefore justified in writing
\begin{align}
\widetilde{q}(o,s|\pi) = p(o|s)q(s|\pi) = q(o|s)q(s|\pi) = q(o,s|\pi)
\end{align}
without violating the definition of EFE. The final expression then becomes
\begin{align}
EFE = H[q(o,s|\pi)||p(o,s|\pi)] -H[q(s | \pi)] - \mathbf{E}_{\widetilde{q}(o,s|\pi)} \log [C_t]
\end{align}

## Conditional Entropy
Conditional entropy can be defined as 
\begin{align}
H[p(x|y)] = \sum_y p(y) \sum_x p(x|y) \log p(x|y) = \sum_{x,y} p(x,y) \log p(x|y)
\end{align}
Multiplying and dividing by $p(y)$
\begin{align}
H[p(x|y)] = \sum_{x,y} p(x,y) \log \frac{p(y)}{p(x,y)}
\end{align}
If we assume the optimization of EFE was successful - and $p=q$ (which is not unreasonable given that there are no data constraints) - we can substitute the relevant terms into the definition of conditional entropy and write
\begin{align}
H[p(x|y)] = \sum_{x,y} q(x,y) \log \frac{q(y)}{p(x,y)} = \sum_{o,s} q(o,s|\pi) \log \frac{q(s|\pi)}{p(o,s|\pi)} = H[p(o|s)]
\end{align}
Substituting back into the full expression we find that EFE can be interpreted as optimizing an expected cost $\mathbf{E}_{\widetilde{q}(o,s|\pi)} [C_t]$ and a term that approaches the conditional entropy of observations $o$ given states $s$. The expected cost drives goal directed behaviour and regularization by an approximate conditional entropy drives exploration. It is also in agreement with the equations used to simulate the canonical T-maze experiment originally presented in \cite{friston 2015} Now we turn towards implementing an example.

## Experiment
We will simulate a linear Gaussian dynamical system with additive control. This is very similar in structure to a Kalman filter with a few keyt differences. First of all, EFE is only applicable for future timepoints. This means there are no data points available for the model. In the absence of data we can rely on the predictive distribution of our generative model to provide a prior over future trajectories.\
Initially the posterior predictive distribution will be identical to the recognition density $q$ since there are no data constraints. Treating $q(\pi_t)$ as a parameter vector we can then optimize EFE to trade off between expected cost and approximate conditional entropy. We assume $p(\pi_t) = 0$ for all $t$ and define initial state
\begin{align}
p(s_0) = q(s_0) = \mathcal{N}([1,1],I)
\end{align}
with transition matrix A and process noise Q
\begin{align}
A = I \\
Q = I * 0.1
\end{align}
emission matrix H and observation noise R
\begin{align}
H = R = I
\end{align}
and goal prior $p(o|m)$
\begin{align}
\mathcal{N}([2,2],I)
\end{align}
This choice of goal prior a feature of EFE by making the expected cost a crossentropy

In [2]:
# TODO: Calculate cross entropy properly
function cross_entropy_gaussians(p,q)
    # Calculates cross entropy of 2 MvNormals
    kl_mvgaussians(p,q) + entropy(q)
end

function kl_mvgaussians(q,p)
    if size(p) != size(q)
        println("Dimensionality mismatch!")
    else

    Σ_q = Matrix(q.Σ)
    Σ_p = Matrix(p.Σ)
    # Fix division by 0 and make sure dim of n is correct. Also having to use "Matrix()" is bad.
    return 0.5* (
	  log(det(Σ_q) / det(Σ_p)) -
	  size(p)[1] +
	  tr(inv(Σ_q)* Σ_p) +
      transpose(q.μ - p.μ) * inv(Σ_q) * (q.μ - p.μ)
        )
    end
end

# Computes EFE for known transition and emission matrices, up to a time horizon T
function compute_EFE(p_s_0,q_s_0,p_o_given_m,π_t,A,Q,H,R,T)
    # p_s_0 = Prior over initial state
    # q_s_0 = initial recognition dist over initial state
    # p_o_given_m = p(o|m), goal prior
    # π_t = q(π), vector of actions. Same dimensionality as states
    # A = Transition matrix
    # Q = Process noise
    # H = Emission matrix
    # R = Emission noise
 
    # Start loss at 0
    loss = 0
    
    for t ∈ 1:T
        # Compute next states q(s_t|s_{t-1}, π_t). Known dynamics A, so only param is action π_t
        q_s_t = MvNormal(A*q_s_0.μ + π_t[t],A*q_s_0.Σ*transpose(A) + Q)
        
        # Compute prior p(s_t | s_{t-1}). p(π_t) = 0, ie we don't assume any action apriori
        p_s_t = MvNormal(A*p_s_0.μ ,A*p_s_0.Σ*transpose(A) + Q)

        # Compute joint q(o,s | π_t)
        q_os_t = MvNormal([q_s_t.μ;H*q_s_t.μ],
                  [q_s_t.Σ q_s_t.Σ*transpose(H) ;
                   H*q_s_t.Σ H*q_s_t.Σ*transpose(H) + R])

        # Compute joint p(o,s|s_{t-1})
        p_os_t = MvNormal([p_s_t.μ;H*p_s_t.μ],
                  [p_s_t.Σ p_s_t.Σ*transpose(H) ;
                   H*p_s_t.Σ H*p_s_t.Σ*transpose(H) + R])

        # Compute marginal q(o_t)
        q_o = MvNormal(H*q_s_t.μ,H*q_s_t.Σ*transpose(H) + R)

        # Evaluate loss as H[q(o,s|pi) || p(o,s|s_{t-1})] - H[q(s_t)] + H[p(o) || p(o|m)]
        loss += cross_entropy_gaussians(p_os_t,q_os_t) - entropy(q_s_t) + cross_entropy_gaussians(q_o,p_o_given_m)
        
        # Print terms
        println("Approximate conditional entropy: ",cross_entropy_gaussians(p_os_t,q_os_t) - entropy(q_s_t))
        println("Instrumental value: ", cross_entropy_gaussians(q_o,p_o_given_m))
        
        # Update distributions for next timestep
        q_s_0 = q_s_t
        p_s_0 = p_s_t 
    end
    return loss
end


compute_EFE (generic function with 1 method)

In [3]:
# Transition matrix and process noise
A = [1. 0. ; 0. 1.]
Q = [0.1 0. ; 0. 0.1]

# Emission matrix and process noise
H = [1. 0. ; 0. 1.]
R = [0.1 0. ; 0. 0.1]

# Goal prior.
p_o_given_m = MvNormal([2.,2.],[0.01 0.0 ; 0.0 0.01])
q_s_0 = MvNormal([1.;1.],[1.0 0. ;0. 1.0])
p_s_0 = MvNormal([1.;1.],[1.0 0. ;0. 1.0])

# Horizon
T = 2

# Action as a scalar param. Needs extension to distribution
π_t = [[0.5,0.5] for i in 1:T]
;

In [4]:
compute_EFE(p_s_0,q_s_0,p_o_given_m,π_t,A,Q,H,R,T)

Approximate conditional entropy: 0.762564700688027
Instrumental value: 2.236865289869967
Approximate conditional entropy: 1.3686253067486325
Instrumental value: 2.1079336385691443


6.475988935875771